In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
input_file = 'data/sorted_filled_data_cancel.pkl'

In [3]:
df = pd.read_pickle(input_file)
df = df.drop(columns=df.columns[0])
df.head()

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model,Subscription Premium Cancel
0,0006869712ec9841dc36234bce245203,MD,Add Payment Method Success,"{""Payment Method"":""Debit"",""Tokenized Pay"":""""}",2022-01-19 18:47:45,android,samsung,SM-A716U,0
1,0006869712ec9841dc36234bce245203,MD,Subscription Premium,"{""Start Date"":""2022-01-19"",""Renewal Date"":""202...",2022-01-19 18:48:01,android,samsung,SM-A716U,0
2,0006869712ec9841dc36234bce245203,MD,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""...",2022-01-19 18:48:08,android,samsung,SM-A716U,0
3,0006869712ec9841dc36234bce245203,MD,Account History Transaction Details,"{""Navigated From"":""Account History"",""Status"":""...",2022-01-19 18:48:44,android,samsung,SM-A716U,0
4,0006869712ec9841dc36234bce245203,MD,Add Vehicle Success,"{""Make"":""Chevrolet"",""Model"":""Express 3500"",""Co...",2022-01-19 18:50:34,android,samsung,SM-A716U,0


In [4]:
import json

df['event_attributes'] = df['event_attributes'].apply(json.loads)

event_attributes = df['event_attributes'].to_list()
event_attributes_df = pd.DataFrame(event_attributes)

enriched_df = df.join(event_attributes_df)
enriched_df = enriched_df.drop(columns='event_attributes')

enriched_df.head()

,userid,user_state,event_name,event_created_date,event_platform,device_manufacture,device_model,Subscription Premium Cancel,Payment Method,Tokenized Pay,...,Error Code,Error Message,Screen,Transaction type,Id,Amount,Description,Google Map Installed,Selected Route Type,Transaction Type
0,0006869712ec9841dc36234bce245203,MD,Add Payment Method Success,2022-01-19 18:47:45,android,samsung,SM-A716U,0,Debit,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0006869712ec9841dc36234bce245203,MD,Subscription Premium,2022-01-19 18:48:01,android,samsung,SM-A716U,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0006869712ec9841dc36234bce245203,MD,Account History Transaction Details,2022-01-19 18:48:08,android,samsung,SM-A716U,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0006869712ec9841dc36234bce245203,MD,Account History Transaction Details,2022-01-19 18:48:44,android,samsung,SM-A716U,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0006869712ec9841dc36234bce245203,MD,Add Vehicle Success,2022-01-19 18:50:34,android,samsung,SM-A716U,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
id__event_name = enriched_df.groupby('userid')['event_name'].apply(list)
id__event_name.head()

userid
0006869712ec9841dc36234bce245203    [Add Payment Method Success, Subscription Prem...
00596aa4edee32815aaf4aad659c484e    [Sign Up Success, Add Vehicle Success, Add Pay...
007b54670d72df309afd87befcbb63db    [Sign Up Success, Add Vehicle Success, Add Pay...
0136a8eeca5d16ac1a521dd0e27ecec0    [Email Confirmation Success, Add Vehicle Succe...
01529f53aaba5b512168d432d5650556    [Sign Up Success, Add Vehicle Break, Add Vehic...
Name: event_name, dtype: object

In [17]:
id__cancel = enriched_df.groupby('userid')['Subscription Premium Cancel'].apply(max)
id__cancel.head()

userid
0006869712ec9841dc36234bce245203    0
00596aa4edee32815aaf4aad659c484e    0
007b54670d72df309afd87befcbb63db    0
0136a8eeca5d16ac1a521dd0e27ecec0    0
01529f53aaba5b512168d432d5650556    0
Name: Subscription Premium Cancel, dtype: int64

In [21]:
cancel_event_df = pd.concat([id__event_name, id__cancel], axis=1)
cancel_event_df.head()

,event_name,Subscription Premium Cancel
userid,,
0006869712ec9841dc36234bce245203,"[Add Payment Method Success, Subscription Prem...",0
00596aa4edee32815aaf4aad659c484e,"[Sign Up Success, Add Vehicle Success, Add Pay...",0
007b54670d72df309afd87befcbb63db,"[Sign Up Success, Add Vehicle Success, Add Pay...",0
0136a8eeca5d16ac1a521dd0e27ecec0,"[Email Confirmation Success, Add Vehicle Succe...",0
01529f53aaba5b512168d432d5650556,"[Sign Up Success, Add Vehicle Break, Add Vehic...",0


In [27]:
events = cancel_event_df['event_name']

all_events = []
for row in events:
    for event in row:
        all_events.append(event)

all_events = list(set(all_events))
all_events

['Sign Up Error',
 'Order',
 'Wallet Opened',
 'Calculator Used',
 'Account History Transaction Details',
 'Sign Up Success',
 'Subscription Premium',
 'Add Vehicle Success',
 'Chat Conversation Started',
 'Add Payment Method Success',
 'Add Vehicle Failed',
 'Email Confirmation Success',
 'Account Setup Profile Skip',
 'Account Setup Skip',
 'Calculator View',
 'Subscription Premium Renew',
 'Reset Password Set',
 'Subscription Premium Cancel',
 'Add Payment Method Failed',
 'Sign Out',
 'Add Vehicle Break',
 'Transaction Refund',
 'Chat Conversation Opened']

In [63]:
all_events_encoded = pd.DataFrame(columns=all_events)

for _, row in cancel_event_df.iterrows():
    events = row['event_name']
    events_encoded = [0] * len(all_events)
    for event in events:
        event_index = all_events.index(event)
        events_encoded[event_index] = 1

    all_events_encoded.loc[len(all_events_encoded)] = events_encoded

all_events_encoded.head()

,Sign Up Error,Order,Wallet Opened,Calculator Used,Account History Transaction Details,Sign Up Success,Subscription Premium,Add Vehicle Success,Chat Conversation Started,Add Payment Method Success,...,Account Setup Skip,Calculator View,Subscription Premium Renew,Reset Password Set,Subscription Premium Cancel,Add Payment Method Failed,Sign Out,Add Vehicle Break,Transaction Refund,Chat Conversation Opened
0,0,0,0,0,1,0,1,1,0,1,...,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,1,1,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0
